# Descarga de excels

In [ ]:
from datetime import datetime,timedelta
Inicio_programa=datetime.now()

# Importo los paquetes

In [ ]:
import os
import tabula
import re
from openpyxl import Workbook,load_workbook
import pandas as pd
import shutil
from natsort import natsorted

In [ ]:
path_local="/home/rodrigo/Scrapper_Sueldos_Municipales"

# Verificación de carpetas

In [ ]:
path_folder_pdf=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/pdf"
lista_folder_pdf=natsorted(os.listdir(path_folder_pdf))
for i in lista_folder_pdf:
    if re.search("Sueldos",i) is None:
        lista_folder_pdf.remove(i)

In [ ]:
lista_folder_pdf

In [ ]:
index_carpeta_a_modificar=-1

In [ ]:
folder_pdf_a_convertir=lista_folder_pdf[index_carpeta_a_modificar]

# Crea la carpeta donde se van a guardar los excel en crudo

In [ ]:
path_folder_pdf_a_convertir=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/pdf/"+folder_pdf_a_convertir
Lista_pdf_nuevos=natsorted(os.listdir(path_folder_pdf_a_convertir))

Lista_nombres_secretarias=[re.sub("\.pdf","",i) for i in Lista_pdf_nuevos]


try :
    os.mkdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza")
    for secretaria in Lista_nombres_secretarias:
        os.mkdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza"+"/"+secretaria)
except :
    pass

for ubicacion_secretaria,nombre_secretaria in enumerate(Lista_pdf_nuevos):
    variable = True
    n=0
    df = tabula.read_pdf(path_folder_pdf_a_convertir+"/"+nombre_secretaria, pages = "all")
    while (variable == True):
        try :
            df[n].to_excel(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza"+"/"+Lista_nombres_secretarias[ubicacion_secretaria]+"/"+Lista_nombres_secretarias[ubicacion_secretaria]+"_imperfecto_"+str(n)+".xlsx")
            n+=1
        except :
            variable=False

In [ ]:
carpeta_especifica=re.sub("_Sueldos","",folder_pdf_a_convertir)
carpeta_especifica

# Acomodo de excel sucios

In [ ]:
# Estos 2 nombres no me gustan
path_carpetas_excels_sucios=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza"
carpetas_excels_sucios=natsorted(os.listdir(path_carpetas_excels_sucios))

for carpeta_secretaria in carpetas_excels_sucios:
    path_excels_sin_modificar= path_carpetas_excels_sucios+"/"+ carpeta_secretaria
    archivos_pdf=natsorted(os.listdir(path_excels_sin_modificar))
    n=0
    for nombre_archivo_pdf in archivos_pdf:
        wb = load_workbook(filename=path_excels_sin_modificar+"/"+nombre_archivo_pdf)
        sheet= wb.active
        sheet.delete_cols(1)
        celda_1=sheet.cell(row=1, column=1).value
        celda_2=sheet.cell(row=1, column=2).value
        if re.search("[a-z]",celda_1) is not None :
            os.remove(path_excels_sin_modificar+"/"+nombre_archivo_pdf)
            n+=1
        elif re.search("[0-9]",celda_1) is not None and re.search("Unnamed: 0",celda_2) is not None:
            inicio=re.search("[0-9]",celda_1).span()[0]
            nombre=celda_1[:inicio]
            numero=celda_1[inicio:]
            sheet["A1"] = nombre
            sheet["B1"] = numero
            sheet.insert_rows(1)
            sheet["A1"],sheet["B1"],sheet["C1"],sheet["D1"],sheet["E1"],sheet["F1"]="apellido_y_nombre","sueldo_bruto","sueldo_neto","costo_laboral","planta","funcion"
            wb.save(filename=path_excels_sin_modificar+"/"+nombre_archivo_pdf)
            os.rename(path_excels_sin_modificar+"/"+nombre_archivo_pdf , path_excels_sin_modificar+"/"+carpeta_secretaria+"_perfecto_"+str(n)+".xlsx")
            n+=1
        else :
            sheet.insert_rows(1)
            sheet["A1"],sheet["B1"],sheet["C1"],sheet["D1"],sheet["E1"],sheet["F1"]="apellido_y_nombre","sueldo_bruto","sueldo_neto","costo_laboral","planta","funcion"
            wb.save(filename=path_excels_sin_modificar+"/"+nombre_archivo_pdf)
            os.rename(path_excels_sin_modificar+"/"+nombre_archivo_pdf,path_excels_sin_modificar+"/"+carpeta_secretaria+"_perfecto_"+str(n)+".xlsx")
            n+=1

# Crea la carpeta donde se almacenan los excel

In [ ]:
path_carpeta_sueldos_nuevos=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos"
try:
    os.mkdir(path_carpeta_sueldos_nuevos)
except:
    pass

nombres_secciones_secretarias=natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/secciones"))
#print(nombres_secciones_secretarias)
for secretaria in nombres_secciones_secretarias:
    try:
        secretaria_sin_extension=re.sub(".txt","",secretaria)
        os.mkdir(os.path.join(path_carpeta_sueldos_nuevos,secretaria_sin_extension))
    except:
        pass
    path_secretarias=os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/secciones",secretaria)
    with open(path_secretarias,"r", encoding='utf8') as f:
        secciones=f.readlines()
    for seccion in secciones:
        try:
            seccion_sin_extension=re.sub(".txt","",seccion)
            os.mkdir(os.path.join(os.path.join(path_carpeta_sueldos_nuevos,secretaria_sin_extension),seccion_sin_extension))
        except Exception as e:
            print(e)

# Re-acomodo los excel en cada subcarpeta

In [ ]:
lista_secretarias_sin_excepciones=["salario_ambiente",
           "salario_control",
           "salario_deporte_turismo",
           "salario_hacienda",
           "salario_ilar",
           "salario_modernizacion",
           "salario_movilidad",
           "salario_obras",
           "salario_salud",
           "salario_spv"]

for secretaria in lista_secretarias_sin_excepciones:
    path_limpieza_secretaria=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/"+secretaria
    nombre_ultimo_archivo_secretaria=re.sub(".xlsx","",natsorted(os.listdir(path_limpieza_secretaria))[-1])
    lista_valores=[*range(int(re.findall(r'\d+',nombre_ultimo_archivo_secretaria)[0])+1)]
    path_secretaria=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/"+secretaria

    lista_secretaria=natsorted(os.listdir(path_secretaria))

    parametro_de_referencia=0
    numero_archivo=0
    for archivo_para_localizar in natsorted(os.listdir(path_limpieza_secretaria)):
        archivo_para_extraer_numero=re.sub(".xlsx","",archivo_para_localizar)
        numero=re.search("[0-999]",archivo_para_extraer_numero)
        inicio_del_numero=numero.span()[0]
        numero_extraido=int(archivo_para_extraer_numero[inicio_del_numero:])
        if numero_extraido == lista_valores[parametro_de_referencia]:
            shutil.move(os.path.join(path_limpieza_secretaria,archivo_para_localizar),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo_para_localizar))
        else:
            for k in natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba")):
                shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",k),os.path.join(path_secretaria,lista_secretaria[numero_archivo]))
            numero_archivo+=1

            #print("Aca deberia ir una funcion que mande todos los excel recogidos dentro de la carpeta\n")

            shutil.move(os.path.join(path_limpieza_secretaria,archivo_para_localizar),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo_para_localizar))

            parametro_de_referencia+=1
        parametro_de_referencia+=1

    lista_ultimos_archivos=natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))
    for archivo in lista_ultimos_archivos:
        shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo),os.path.join(path_secretaria,lista_secretaria[-1]))

# Manejo de excepciones

# Salario Cultura

In [ ]:
path_limpieza_secretaria=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/salario_cultura"

path_secretaria=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_cultura"

lista_secretaria=natsorted(os.listdir(path_secretaria))

shutil.move(os.path.join(path_limpieza_secretaria,"salario_cultura_perfecto_0.xlsx"),os.path.join(os.path.join(path_secretaria,lista_secretaria[0]),"salario_cultura_perfecto_0.xlsx"))

nombre_ultimo_archivo_secretaria=re.sub(".xlsx","",natsorted(os.listdir(path_limpieza_secretaria))[-1])
lista_valores=[*range(1,int(re.findall(r'\d+',nombre_ultimo_archivo_secretaria)[0])+1)]

parametro_de_referencia=0
numero_archivo=1
for archivo_para_localizar in natsorted(os.listdir(path_limpieza_secretaria)):
    archivo_para_extraer_numero=re.sub(".xlsx","",archivo_para_localizar)
    numero=re.search("[0-999]",archivo_para_extraer_numero)
    inicio_del_numero=numero.span()[0]
    numero_extraido=int(archivo_para_extraer_numero[inicio_del_numero:])
    if numero_extraido == lista_valores[parametro_de_referencia]:
        shutil.move(os.path.join(path_limpieza_secretaria,archivo_para_localizar),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo_para_localizar))
    else:
        for k in natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba")):
            shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",k),os.path.join(path_secretaria,lista_secretaria[numero_archivo]))
        numero_archivo+=1

        #print("Aca deberia ir una funcion que mande todos los excel recogidos dentro de la carpeta\n")

        shutil.move(os.path.join(path_limpieza_secretaria,archivo_para_localizar),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo_para_localizar))

        parametro_de_referencia+=1
    parametro_de_referencia+=1

#try:
#    lista_ultimos_archivos=natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))
#    for archivo in lista_ultimos_archivos:
#        shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo),os.path.join(path_secretaria,lista_secretaria[-1]))
#except:
#    pass

# Para el caso 2021_01

#shutil.move(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba/salario_cultura_perfecto_92.xlsx",os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_cultura/40_TEATRO_LA_COMEDIA","salario_cultura_perfecto_92.xlsx"))

#try:
#    lista_ultimos_archivos=natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))
#    for archivo in lista_ultimos_archivos:
#        shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo),os.path.join(path_secretaria,lista_secretaria[-2]))
#except:
#    pass



# Para el caso 2021_04

shutil.move(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba/salario_cultura_perfecto_95.xlsx",os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_cultura/"+lista_secretaria[-1],"salario_cultura_perfecto_95.xlsx"))

try:
    lista_ultimos_archivos=natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))
    for archivo in lista_ultimos_archivos:
        shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo),os.path.join(path_secretaria,lista_secretaria[-2]))
except:
    pass

# Salario Desarrollo Economico

In [ ]:
path_limpieza_secretaria=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/salario_desarrollo_economico"

path_secretaria=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_desarrollo_economico"

lista_secretaria=natsorted(os.listdir(path_secretaria))


shutil.move(os.path.join(path_limpieza_secretaria,"salario_desarrollo_economico_perfecto_0.xlsx"),os.path.join(os.path.join(path_secretaria,lista_secretaria[0]),"salario_desarrollo_economico_perfecto_0.xlsx"))
shutil.move(os.path.join(path_limpieza_secretaria,"salario_desarrollo_economico_perfecto_1.xlsx"),os.path.join(os.path.join(path_secretaria,lista_secretaria[1]),"salario_desarrollo_economico_perfecto_1.xlsx"))

nombre_ultimo_archivo_secretaria=re.sub(".xlsx","",natsorted(os.listdir(path_limpieza_secretaria))[-1])
lista_valores=[*range(2,int(re.findall(r'\d+',nombre_ultimo_archivo_secretaria)[0])+1)]

parametro_de_referencia=0
numero_archivo=1
for archivo_para_localizar in natsorted(os.listdir(path_limpieza_secretaria)):
    archivo_para_extraer_numero=re.sub(".xlsx","",archivo_para_localizar)
    numero=re.search("[0-999]",archivo_para_extraer_numero)
    inicio_del_numero=numero.span()[0]
    numero_extraido=int(archivo_para_extraer_numero[inicio_del_numero:])
    if numero_extraido == lista_valores[parametro_de_referencia]:
        shutil.move(os.path.join(path_limpieza_secretaria,archivo_para_localizar),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo_para_localizar))
    else:
        for k in natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba")):
            shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",k),os.path.join(path_secretaria,lista_secretaria[numero_archivo]))
        numero_archivo+=1

        #print("Aca deberia ir una funcion que mande todos los excel recogidos dentro de la carpeta\n")

        shutil.move(os.path.join(path_limpieza_secretaria,archivo_para_localizar),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo_para_localizar))

        parametro_de_referencia+=1
    parametro_de_referencia+=1

try:
    lista_ultimos_archivos=natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))
    for archivo in lista_ultimos_archivos:
        shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo),os.path.join(path_secretaria,lista_secretaria[-1]))
except:
    pass

# Salario desarrollo humano

In [ ]:
path_limpieza_secretaria=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/salario_desarrollo_humano"

path_secretaria=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_desarrollo_humano"

lista_secretaria=natsorted(os.listdir(path_secretaria))


shutil.move(os.path.join(path_limpieza_secretaria,"salario_desarrollo_humano_perfecto_0.xlsx"),os.path.join(os.path.join(path_secretaria,lista_secretaria[0]),"salario_desarrollo_humano_perfecto_0.xlsx"))
shutil.move(os.path.join(path_limpieza_secretaria,"salario_desarrollo_humano_perfecto_1.xlsx"),os.path.join(os.path.join(path_secretaria,lista_secretaria[1]),"salario_desarrollo_humano_perfecto_1.xlsx"))

nombre_ultimo_archivo_secretaria=re.sub(".xlsx","",natsorted(os.listdir(path_limpieza_secretaria))[-1])
lista_valores=[*range(2,int(re.findall(r'\d+',nombre_ultimo_archivo_secretaria)[0])+1)]

parametro_de_referencia=0
numero_archivo=2
for archivo_para_localizar in natsorted(os.listdir(path_limpieza_secretaria)):
    archivo_para_extraer_numero=re.sub(".xlsx","",archivo_para_localizar)
    numero=re.search("[0-999]",archivo_para_extraer_numero)
    inicio_del_numero=numero.span()[0]
    numero_extraido=int(archivo_para_extraer_numero[inicio_del_numero:])
    if numero_extraido == lista_valores[parametro_de_referencia]:
        shutil.move(os.path.join(path_limpieza_secretaria,archivo_para_localizar),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo_para_localizar))
    else:
        for k in natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba")):
            shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",k),os.path.join(path_secretaria,lista_secretaria[numero_archivo]))
        numero_archivo+=1

        #print("Aca deberia ir una funcion que mande todos los excel recogidos dentro de la carpeta\n")

        shutil.move(os.path.join(path_limpieza_secretaria,archivo_para_localizar),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo_para_localizar))

        parametro_de_referencia+=1
    parametro_de_referencia+=1

try:
    lista_ultimos_archivos=natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))
    for archivo in lista_ultimos_archivos:
        shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo),os.path.join(path_secretaria,lista_secretaria[-1]))
except:
    pass

# Salario Genero

In [ ]:
path_limpieza_secretaria=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/salario_genero_ddhh"

path_secretaria=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_genero_ddhh"

lista_secretaria=natsorted(os.listdir(path_secretaria))

shutil.move(os.path.join(path_limpieza_secretaria,"salario_genero_ddhh_perfecto_0.xlsx"),os.path.join(os.path.join(path_secretaria,lista_secretaria[0]),"salario_genero_ddhh_perfecto_0.xlsx"))

nombre_ultimo_archivo_secretaria=re.sub(".xlsx","",natsorted(os.listdir(path_limpieza_secretaria))[-1])
lista_valores=[*range(1,int(re.findall(r'\d+',nombre_ultimo_archivo_secretaria)[0])+1)]

parametro_de_referencia=0
numero_archivo=1
for archivo_para_localizar in natsorted(os.listdir(path_limpieza_secretaria)):
    archivo_para_extraer_numero=re.sub(".xlsx","",archivo_para_localizar)
    numero=re.search("[0-999]",archivo_para_extraer_numero)
    inicio_del_numero=numero.span()[0]
    numero_extraido=int(archivo_para_extraer_numero[inicio_del_numero:])
    if numero_extraido == lista_valores[parametro_de_referencia]:
        shutil.move(os.path.join(path_limpieza_secretaria,archivo_para_localizar),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo_para_localizar))
    else:
        for k in natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba")):
            shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",k),os.path.join(path_secretaria,lista_secretaria[numero_archivo]))
        numero_archivo+=1

        #print("Aca deberia ir una funcion que mande todos los excel recogidos dentro de la carpeta\n")

        shutil.move(os.path.join(path_limpieza_secretaria,archivo_para_localizar),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo_para_localizar))

        parametro_de_referencia+=1
    parametro_de_referencia+=1

try:
    lista_ultimos_archivos=natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))
    for archivo in lista_ultimos_archivos:
        shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo),os.path.join(path_secretaria,lista_secretaria[-1]))
except:
    pass

# Salario Gobierno

In [ ]:
path_limpieza_secretaria=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/salario_gobierno"

path_secretaria=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_gobierno"

lista_secretaria=natsorted(os.listdir(path_secretaria))

shutil.move(os.path.join(path_limpieza_secretaria,"salario_gobierno_perfecto_0.xlsx"),os.path.join(os.path.join(path_secretaria,lista_secretaria[0]),"salario_gobierno_perfecto_0.xlsx"))
shutil.move(os.path.join(path_limpieza_secretaria,"salario_gobierno_perfecto_1.xlsx"),os.path.join(os.path.join(path_secretaria,lista_secretaria[1]),"salario_gobierno_perfecto_1.xlsx"))

nombre_ultimo_archivo_secretaria=re.sub(".xlsx","",natsorted(os.listdir(path_limpieza_secretaria))[-1])
lista_valores=[*range(2,int(re.findall(r'\d+',nombre_ultimo_archivo_secretaria)[0])+1)]

parametro_de_referencia=0
numero_archivo=2
for archivo_para_localizar in natsorted(os.listdir(path_limpieza_secretaria)):
    archivo_para_extraer_numero=re.sub(".xlsx","",archivo_para_localizar)
    numero=re.search("[0-999]",archivo_para_extraer_numero)
    inicio_del_numero=numero.span()[0]
    numero_extraido=int(archivo_para_extraer_numero[inicio_del_numero:])
    if numero_extraido == lista_valores[parametro_de_referencia]:
        shutil.move(os.path.join(path_limpieza_secretaria,archivo_para_localizar),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo_para_localizar))
    else:
        for k in natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba")):
            shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",k),os.path.join(path_secretaria,lista_secretaria[numero_archivo]))
        numero_archivo+=1

        #print("Aca deberia ir una funcion que mande todos los excel recogidos dentro de la carpeta\n")

        shutil.move(os.path.join(path_limpieza_secretaria,archivo_para_localizar),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo_para_localizar))

        parametro_de_referencia+=1
    parametro_de_referencia+=1

try:
    lista_ultimos_archivos=natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))
    for archivo in lista_ultimos_archivos:
        shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo),os.path.join(path_secretaria,lista_secretaria[-1]))
except:
    pass

# Salario intendencia

In [ ]:
path_limpieza_secretaria=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/salario_intendencia"

path_secretaria=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_intendencia"

lista_secretaria=natsorted(os.listdir(path_secretaria))

shutil.move(os.path.join(path_limpieza_secretaria,"salario_intendencia_perfecto_0.xlsx"),os.path.join(os.path.join(path_secretaria,lista_secretaria[0]),"salario_intendencia_perfecto_0.xlsx"))
shutil.move(os.path.join(path_limpieza_secretaria,"salario_intendencia_perfecto_1.xlsx"),os.path.join(os.path.join(path_secretaria,lista_secretaria[1]),"salario_intendencia_perfecto_1.xlsx"))

nombre_ultimo_archivo_secretaria=re.sub(".xlsx","",natsorted(os.listdir(path_limpieza_secretaria))[-1])
lista_valores=[*range(3,int(re.findall(r'\d+',nombre_ultimo_archivo_secretaria)[0])+1)]

parametro_de_referencia=0
numero_archivo=2
for archivo_para_localizar in natsorted(os.listdir(path_limpieza_secretaria)):
    archivo_para_extraer_numero=re.sub(".xlsx","",archivo_para_localizar)
    numero=re.search("[0-999]",archivo_para_extraer_numero)
    inicio_del_numero=numero.span()[0]
    numero_extraido=int(archivo_para_extraer_numero[inicio_del_numero:])
    if numero_extraido == lista_valores[parametro_de_referencia]:
        shutil.move(os.path.join(path_limpieza_secretaria,archivo_para_localizar),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo_para_localizar))
    else:
        for k in natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba")):
            shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",k),os.path.join(path_secretaria,lista_secretaria[numero_archivo]))
        numero_archivo+=1

        #print("Aca deberia ir una funcion que mande todos los excel recogidos dentro de la carpeta\n")

        shutil.move(os.path.join(path_limpieza_secretaria,archivo_para_localizar),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo_para_localizar))

        parametro_de_referencia+=1
    parametro_de_referencia+=1

try:
    lista_ultimos_archivos=natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))
    for archivo in lista_ultimos_archivos:
        shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo),os.path.join(path_secretaria,lista_secretaria[-1]))
except:
    pass

# Salario Planeamiento

In [ ]:
path_limpieza_secretaria=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/salario_planeamiento"

path_secretaria=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_planeamiento"

lista_secretaria=natsorted(os.listdir(path_secretaria))

shutil.move(os.path.join(path_limpieza_secretaria,"salario_planeamiento_perfecto_0.xlsx"),os.path.join(os.path.join(path_secretaria,lista_secretaria[0]),"salario_planeamiento_perfecto_0.xlsx"))
shutil.move(os.path.join(path_limpieza_secretaria,"salario_planeamiento_perfecto_1.xlsx"),os.path.join(os.path.join(path_secretaria,lista_secretaria[1]),"salario_planeamiento_perfecto_1.xlsx"))
shutil.move(os.path.join(path_limpieza_secretaria,"salario_planeamiento_perfecto_2.xlsx"),os.path.join(os.path.join(path_secretaria,lista_secretaria[2]),"salario_planeamiento_perfecto_2.xlsx"))

nombre_ultimo_archivo_secretaria=re.sub(".xlsx","",natsorted(os.listdir(path_limpieza_secretaria))[-1])
lista_valores=[*range(4,int(re.findall(r'\d+',nombre_ultimo_archivo_secretaria)[0])+1)]

# Para el caso 2020_09
#lista_valores=[*range(3,int(re.findall(r'\d+',nombre_ultimo_archivo_secretaria)[0])+1)]

parametro_de_referencia=0
numero_archivo=3

for archivo_para_localizar in natsorted(os.listdir(path_limpieza_secretaria)):
    archivo_para_extraer_numero=re.sub(".xlsx","",archivo_para_localizar)
    numero=re.search("[0-999]",archivo_para_extraer_numero)
    inicio_del_numero=numero.span()[0]
    numero_extraido=int(archivo_para_extraer_numero[inicio_del_numero:])
    if numero_extraido == lista_valores[parametro_de_referencia]:
        shutil.move(os.path.join(path_limpieza_secretaria,archivo_para_localizar),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo_para_localizar))
    else:
        for k in natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba")):
            shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",k),os.path.join(path_secretaria,lista_secretaria[numero_archivo]))
        numero_archivo+=1

        #print("Aca deberia ir una funcion que mande todos los excel recogidos dentro de la carpeta\n")

        shutil.move(os.path.join(path_limpieza_secretaria,archivo_para_localizar),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo_para_localizar))

        parametro_de_referencia+=1
    parametro_de_referencia+=1

try:
    lista_ultimos_archivos=natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))
    for archivo in lista_ultimos_archivos:
        shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",archivo),os.path.join(path_secretaria,lista_secretaria[-1]))
except:
    pass

# Elimina la carpeta "Excels_proceso_limpieza"

In [ ]:
shutil.rmtree(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza")

# Creo los archivos finales

In [ ]:
path_folder_carpeta=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos"
lista_secretarias=natsorted(os.listdir(path_folder_carpeta))
path_carpeta_prueba=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"
for secretaria in lista_secretarias:
    path_folder_secretaria=os.path.join(path_folder_carpeta,secretaria)
    lista_secciones_secretaria=natsorted(os.listdir(path_folder_secretaria))
    for seccion in lista_secciones_secretaria:
        path_folder_secciones=os.path.join(path_folder_secretaria,seccion)
        lista_archivos_secciones=natsorted(os.listdir(path_folder_secciones))
        path_primer_archivo_seccion=os.path.join(path_folder_secciones, lista_archivos_secciones[0])
        path_archivo_final=path_folder_secciones+".xlsx"

        if len(lista_archivos_secciones) == 1:
            shutil.move(path_primer_archivo_seccion,path_archivo_final)
        else:
            shutil.move(path_primer_archivo_seccion,path_carpeta_prueba)
            for archivo in lista_archivos_secciones[1:]:
                shutil.move(os.path.join(path_folder_secciones,archivo),path_carpeta_prueba)
                lista_archivos_carpeta_prueba=natsorted(os.listdir(path_carpeta_prueba))
                path_df_1=os.path.join(path_carpeta_prueba,lista_archivos_carpeta_prueba[0])
                path_df_2=os.path.join(path_carpeta_prueba,lista_archivos_carpeta_prueba[1])
                df_1=pd.read_excel(path_df_1)
                df_2=pd.read_excel(path_df_2)
                df_final=pd.concat([df_1,df_2],ignore_index=True)
                os.remove(path_df_1)
                os.remove(path_df_2)
                df_final.to_excel(path_df_1,index=False,header=True)
            path_ubicacion_archivo_final=os.path.join(path_carpeta_prueba,lista_archivos_carpeta_prueba[0])
            shutil.move(path_ubicacion_archivo_final,path_archivo_final)

# Elimina las carpetas

In [ ]:
for secretarias in os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos"):
    for archivos_secretaria in os.listdir(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos",secretarias)):
        try:
            os.rmdir(os.path.join(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos",secretarias),archivos_secretaria))
        except:
            pass

# Reviso la columna 1 de todos los excel y muevo los numeros a la segunda columna

In [ ]:
path=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos"
lista_secretarias=natsorted(os.listdir(path))
for secretaria in lista_secretarias:
    path_secretaria=os.path.join(path,secretaria)
    lista_archivos=natsorted(os.listdir(path_secretaria))
    for archivo in lista_archivos:
        path_archivo=os.path.join(path_secretaria,archivo)
        df=pd.read_excel(path_archivo)
        final=df.shape[0]
        workbook= load_workbook(filename=path_archivo)
        sheet= workbook.active
        for fila in range(1,final+2):
            celda=sheet.cell(row=fila, column=1).value
            try:
                inicio_numero=re.search("[0-9]",celda).span()[0]
                nombre=celda[:inicio_numero]
                numero=celda[inicio_numero:]
                sheet["A"+str(k)] = nombre
                sheet["B"+str(k)] = numero
            except:
                pass
        workbook.save(filename=path_archivo)

# Reemplazo las comas por puntos

In [ ]:
path=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos"
for secretaria in natsorted(os.listdir(path)):
    lista_secretaria=natsorted(os.listdir(os.path.join(path,secretaria)))
    for archivo in lista_secretaria:
        path_archivo=os.path.join(os.path.join(path,secretaria),archivo)
        df=pd.read_excel(path_archivo)
        final=df.shape[0]
        workbook= load_workbook(filename=path_archivo)
        sheet= workbook.active
        for fila in range(1,final+2):
            celda_1=sheet.cell(row=fila, column=2).value
            celda_2=sheet.cell(row=fila, column=3).value
            celda_3=sheet.cell(row=fila, column=4).value
            try:
                sheet["B"+str(k)]=re.sub(",",".",celda_1)
                sheet["C"+str(k)]=re.sub(",",".",celda_2)
                sheet["D"+str(k)]=re.sub(",",".",celda_3)
            except:
                pass
        workbook.save(filename=path_archivo)

# Esto revisa unos numeros extras que salen random luego de los 2 decimales

In [ ]:
path=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos"
lista_secretarias=natsorted(os.listdir(path))
for columna in [2,3,4]:
    for secretaria in lista_secretarias:
        lista_archivos=natsorted(os.listdir(os.path.join(path,secretaria)))
        for archivo in lista_archivos:
            path_archivo=os.path.join(os.path.join(path,secretaria),archivo)
            df=pd.read_excel(path_archivo)
            final=df.shape[0]
            workbook= load_workbook(filename=path_archivo)
            sheet= workbook.active
            for fila in range(1,final+2):
                celda=str(sheet.cell(row=fila, column=columna).value)
                if len(re.findall("\.",celda))>1:
                    final=re.search("\.",celda).span()[1]
                    sheet["B"+str(fila)]=celda[:final+2]
                else:
                    pass
            workbook.save(filename=path_archivo)

# Mido cuanto tardo el programa (aproximadamente)

In [ ]:
Finalizacion_programa=datetime.now()
tiempo=Finalizacion_programa-Inicio_programa
tiempo_medido=str(timedelta(seconds=tiempo.seconds))[2:]
tiempo_medido